In [0]:
import autoreload

%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import time
import os
from collections import OrderedDict
from torch.utils.data import Subset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Stuff')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=8,shuffle=False, num_workers=2)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
def trainset_node_split(dataset, N, seed = 0):
    np.random.seed(seed)
    a = np.arange(len(dataset))
    np.random.shuffle(a)
    datasets = {}
    size = int(len(dataset)/N)
    for i in range(N):
        datasets[i] = Subset(dataset, a[i*size:(i+1)*size].tolist())
    return datasets

In [0]:
def count_correct(outputs, labels):
    """ count correct predictions """

    if isinstance(criterion, nn.BCELoss):
        predicted = (outputs > 0.5).to(dtype=torch.int64)
        labels = (labels  > 0.5).to(dtype=torch.int64)
    elif isinstance(criterion, nn.CrossEntropyLoss):
        _, predicted = outputs.max(1)
    else:
        print('Error in criterion')
        raise ValueError

    correct = (predicted == labels).sum().item()

    return correct

def forward_test(model, loader, chosen_device):
    """ forwards test samples and calculates the test accuracy """

    model.to(chosen_device)
    running_losses = 0.0
    correct = 0
    total = 0
    count_batches = 0

    with torch.no_grad():

        for batch_idx, sample in enumerate(loader):

            inputs, labels = sample[0].to(chosen_device), sample[1].to(chosen_device)
            #inputs, labels = sample[0], sample[1]
            outputs = model(inputs)
            
            loss = criterion(outputs,labels)
            
            count_batches += 1

            running_losses += loss.item()

            correct += count_correct(outputs, labels)
            total += labels.size(0)


    test_acc  = 100.0 * correct / total
    test_loss = running_losses / total

    print('\n=> Test acc  : {:7.3f}%'.format(test_acc))

    return test_acc, test_loss

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        #self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        #self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 4 * 4, 64)
        #self.fc2 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(64, 10)
        self.init_weights()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x

    def init_weights(self):
        
        for m in self.modules():

            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out',nonlinearity='relu')

            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
                torch.nn.init.xavier_uniform_(m.weight)

In [0]:
class Node():
    """Node(Choco_Gossip): x_i(t+1) = x_i(t) + gamma*Sum(w_ij*[xhat_j(t+1) - xhat_i(t+1)])"""
    
    def __init__(self, gamma, loader, batch_size, dataset, model, criterion, chosen_device):
        

        self.batch_size = batch_size
        self.neighbors = []
        self.neighbor_wts = {}
        
        self.step_size = gamma

        self.dataset = dataset

        self.dataloader = loader
        
        self.model = model
        self.chosen_device = chosen_device
        
        #self.model.to(self.chosen_device)

        self.x_i = OrderedDict()
        
        self.model_params = []
        for (k,v) in self.model.state_dict().items():
            
            self.model_params.append(k)
            self.x_i[k] = v.clone().detach()
            
        #for a in self.model.parameters():
        #    self.x_i.append(a)
        
        self.criterion = criterion
        if self.dataloader is not None:
            self.dataiter = iter(self.dataloader)
        
    
    def compute_gradient(self, quantizer=None, ):
        """Computes nabla(x_i, samples) and returns estimate after quantization"""
        
        # Sample batch from loader #
        optimizer  = optim.SGD(self.model.parameters(), lr=1e-3)
        #for v in self.model.parameters():
        #  if v.grad is not None:
        #    v.detach_()
        #    v.zero_()

        optimizer.zero_grad()    

        try:
            inputs, targets = self.dataiter.next()
        except StopIteration:
            self.dataiter = iter(self.dataloader)
            inputs, targets = self.dataiter.next()

        inputs, targets = inputs.to(self.chosen_device), targets.to(self.chosen_device) 

        self.model.to(self.chosen_device)
        outputs = self.model(inputs)


        loss = self.criterion(outputs, targets)
        
        #Equivalent to optimizer.zero_grad()
        
        
        loss.backward()
        
        gt = OrderedDict()
        
        
        for k,v in enumerate(self.model.parameters()):
            if v.grad is not None:
                if quantizer is not None:
                    gt[k] = quantizer(v.grad.clone().detach_())
                else:
                    gt[k] = v.grad.clone().detach()
        #optimizer.step()
    
        self.curr_gt = gt
        
        return
    
    def assign_params(self, W):
        """Assign dict W to model"""
        
        with torch.no_grad():
            self.model.load_state_dict(W, strict=False)
        
        return
    

    def update_model(self):
        
        ### Implement Algorithm ###
        
        ## Assign Parameters after obtaining Consensus##
        
        
        self.assign_params(self.x_i)
        
        return


    def calc_node_loss(self, testloader, chosen_device):
        """ loss check """
        
        loss_dict = {}

        #self.model.to(chosen_device)
        trainloader = torch.utils.data.DataLoader(self.dataset, self.batch_size, shuffle=True, num_workers=2)

        running_losses = 0.0
        correct = 0
        total = 0
        count_batches = 0

        with torch.no_grad():

            for batch_idx, sample in enumerate(trainloader):

                inputs, labels = sample[0].to(chosen_device), sample[1].to(chosen_device)
                #inputs, labels = sample[0], sample[1]
                self.model.to(self.chosen_device)
                outputs = self.model(inputs)
                
                loss = criterion(outputs,labels)
                
                count_batches += 1

                running_losses += loss.item()

                correct += count_correct(outputs, labels)
                total += labels.size(0)

                running_losses = running_losses/self.batch_size


        loss_dict["train_acc"]  = 100.0 * correct / total
        loss_dict["train_loss"] = running_losses / total

        running_losses = 0.0
        correct = 0
        total = 0
        count_batches = 0

        with torch.no_grad():

            for batch_idx, sample in enumerate(testloader):

                inputs, labels = sample[0].to(chosen_device), sample[1].to(chosen_device)
                #inputs, labels = sample[0], sample[1]
                self.model.to(self.chosen_device)
                outputs = self.model(inputs)
                
                loss = criterion(outputs,labels)
                
                count_batches += 1

                running_losses += loss.item()

                correct += count_correct(outputs, labels)
                total += labels.size(0)

                running_losses = running_losses/self.batch_size


        loss_dict["test_acc"]  = 100.0 * correct / total
        loss_dict["test_loss"] = running_losses / total

        #print('\n=> Test acc  : {:7.3f}%'.format(test_acc))

        return loss_dict

In [0]:
class Network():
    """Define graph"""
    
    def __init__(self, W, models, datasets, learning_rates, loaders, batch_size, criterion, chosen_device, testloader):
        
        self.adjacency = W
        self.num_nodes = W.shape[0]
        self.chosen_device = chosen_device
        self.batch_size = batch_size
        self.testloader = testloader
        
        self.nodes = OrderedDict()
        
        for i in range(self.num_nodes):
            self.nodes[i] = Node(learning_rates[i], loaders[i], self.batch_size, datasets[i], models[i], criterion, self.chosen_device)
            for j in range(self.num_nodes):
                if(j != i and W[i, j] > 0):
                    self.nodes[i].neighbors.append(j)
                    self.nodes[i].neighbor_wts[j] = W[i, j]
                    

    def consensus_test(self, loader, batch_size):
        """ forwards test samples and calculates the test accuracy """

        #model.to(chosen_device)
        correct = 0
        total = 0
        count_batches = 0

        with torch.no_grad():

            for batch_idx, sample in enumerate(loader):

                inputs, labels = sample[0].to(self.chosen_device), sample[1].to(self.chosen_device)
                #inputs, labels = sample[0], sample[1]
                for i in range(self.num_nodes):
                    self.nodes[i].model.to(self.chosen_device)
                    outputs = self.nodes[i].model(inputs)
                    if(i == 0):
                        consensus = torch.zeros_like(outputs)

                    logits, predicted = outputs.max(1)
                    #print(logits)
                    
                    for i in range(batch_size):
                        consensus[i] = consensus[i]+torch.where(outputs[i].eq(logits[i]), torch.Tensor([1]).to(self.chosen_device), torch.Tensor([0]).to(self.chosen_device))              
                
                logits, final_pred = consensus.max(1)

                batch_correct = (final_pred == labels).sum().item()
                
                correct += batch_correct
                total += labels.size(0)

        test_acc  = 100.0 * correct / total
        #test_loss = running_losses / total

        #print('\n=> Test acc  : {:7.3f}%'.format(test_acc))

        return test_acc
        

    def simulate(self, iterations, epochs):
        
        record_sims = OrderedDict()
        for k in range(self.num_nodes):
          record_sims[k] = []

        for i in range(epochs):
            for j in range(iterations):
                lr = 1e-3
                if((j+1) % 500 == 0 and j != 0):
                  
                  test_acc = self.consensus_test(self.testloader, self.batch_size)
                  for k in range(self.num_nodes):
                    loss_dict = self.nodes[k].calc_node_loss(self.testloader, self.chosen_device)
                    loss_dict["consensus_test"] = test_acc
                    loss_dict["iteration"] = j
                    record_sims[k].append(loss_dict)
                    
                for k in range(self.num_nodes):
                    self.nodes[k].compute_gradient()
                
                
                for l in range(self.num_nodes):
                    for m,param in enumerate(self.nodes[l].model.parameters()):
                        if param.grad is None:
                            continue
                      
                        gt_update = self.nodes[l].curr_gt[m]
                        wt_sum = 1
                        for n in self.nodes[l].neighbors:
                            gt_update= gt_update + self.nodes[l].neighbor_wts[n] *self.nodes[n].curr_gt[m]
                            wt_sum = wt_sum + self.nodes[l].neighbor_wts[n]
                        gt_update = gt_update/wt_sum
                        param.data = param.data - lr*gt_update
                    #self.nodes[str(l)].update_model()

        return record_sims

In [0]:
class Agg_Server():
    
    def __init__(self, W, models, clr_model, learning_rates, clr, loaders, criterion):
        
        self.num_nodes = W.shape[0]
        self.server_weights = W
        
        self.central_server = Node(clr, None, clr_model, criterion)
        
        self.nodes = OrderedDict()
        
        for i in range(self.num_nodes):
            self.nodes[i] = Node(learning_rates[i], loaders[i], models[i], criterion)
            
            if(W[i, 0] > 0):
                self.central_server.neighbors.append(i)
                self.central_server.neighbor_wts[i] = W[i, 0]

            
    def simulate(self, iterations, epochs):
        
        for i in range(epochs):
            for j in range(iterations):
                lr = 1e-3
                if(j % 500 == 0):
                    print(j)
                    
                for k in range(self.num_nodes):
                    self.nodes[k].compute_gradient()
    

                for m,param in enumerate(self.central_server.model.parameters()):
                    
                    for a,n in enumerate(self.central_server.neighbors):
                        
                        if self.nodes[n].curr_gt[m] is None:
                            continue
                        
                        if(a == 0):
                            gt = torch.sign(self.nodes[n].curr_gt[m])
                            gt_update = self.central_server.neighbor_wts[n]*gt
                            wt_sum = self.central_server.neighbor_wts[n]
                        else:
                            gt = torch.sign(self.nodes[n].curr_gt[m])
                            gt_update= gt_update + self.central_server.neighbor_wts[n]*gt
                            wt_sum = wt_sum + self.central_server.neighbor_wts[n]
                        
                    gt_update = gt_update/wt_sum
                    param.data = param.data - lr*gt_update
                    
                
                for a,n in enumerate(self.central_server.neighbors):
                    
                    self.nodes[n].model.load_state_dict(self.central_server.model.state_dict())
                
                    #self.nodes[str(l)].update_model()

## Agg Server

In [37]:
models = [Net() for i in range(7)]
#c_model = Net()
c_lr = 1e-3

criterion = nn.CrossEntropyLoss()

m = trainset_node_split(trainset, 2)

trainloaders = [torch.utils.data.DataLoader(m[i], batch_size=4, shuffle=True, num_workers=2) for i in range(2)]



net = Agg_Server(torch.ones([2,1]),models, c_model, [1e-3,1e-3], c_lr, trainloaders, nn.CrossEntropyLoss())

NameError: ignored

In [0]:
net.simulate(4000,1)

0
500
1000
1500
2000
2500
3000
3500


In [0]:
forward_test(c_model, testloader, device)


=> Test acc  :  78.660%


(78.66, 0.676750244140625)

In [0]:
forward_test(models[0], testloader, device)


=> Test acc  :  91.170%


(91.17, 0.07826509742736816)

In [0]:
forward_test(models[1], testloader, device)


=> Test acc  :  78.000%


(78.0, 0.19203385961055755)

In [0]:
def quantizer_topk(gradient, k = 5):
    absoulte = torch.abs( gradient )
    sign  = torch.sign(gradient)
    values,indices = torch.topk( gradient, k , sorted = False )
    gradient = torch.zeros( *gradient.shape )
    gradient[indices] = values
    #transform gradient to torch
    return gradient*sign

def quantizer_lossy( gradient, k = 5 ):
    norm = torch.norm( gradient )
    absoulte = torch.abs( gradient )
    absoulte = ( absoulte/norm )*k
    floor = torch.floor(gradient)
    random_ceil = torch.rand(*gradient.shape) < ( gradient - floor )
    print( random_ceil )
    floor = ( floor + random_ceil.float() ) * (1/k)
    #rescale
    return (norm) * ( torch.sign(gradient) * floor )
    
a = torch.arange(5, dtype= torch.float)
print(quantizer_lossy(a),  5 )
a = torch.arange( 10 , dtype= torch.float )
a = a - 5
print(quantizer_topk(a))

In [0]:
train_loss = OrderedDict()
test_loss = OrderedDict()

## Ring

In [13]:
W = np.zeros([9,9])
for i in range(9):
  if(i > 0 and i < 8):
    W[i, i+1] = 1
    W[i, i-1] = 1
  elif(i == 0):
    W[i, i+1] = 1
    W[i, 8] = 1
  else:
    W[i, i-1] = 1
    W[i, 0] = 1

print(W)

[[0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [0]:
models = [Net() for i in range(9)]
#c_model = Net()

criterion = nn.CrossEntropyLoss()

m = trainset_node_split(trainset, 9)

trainloaders = [torch.utils.data.DataLoader(m[i], batch_size=8, shuffle=True, num_workers=2) for i in range(9)]

lrs = []

for i in range(9):
  lrs.append(1e-3)
  
#net = Network(W,models,lrs, trainloaders, nn.CrossEntropyLoss())

In [0]:
net = Network(W, models, m, lrs, trainloaders, 8, nn.CrossEntropyLoss(), device, testloader)

a = net.simulate(500, 1)

In [41]:
print(a)

OrderedDict([(0, [{'train_acc': 24.032403240324033, 'train_loss': 4.500357460360147e-05, 'test_acc': 23.66, 'test_loss': 3.2054605530724326e-05, 'consensus_test': 23.04, 'iteration': 499}]), (1, [{'train_acc': 11.281128112811281, 'train_loss': 4.434331395542826e-05, 'test_acc': 12.25, 'test_loss': 3.317279109856046e-05, 'consensus_test': 23.04, 'iteration': 499}]), (2, [{'train_acc': 13.951395139513952, 'train_loss': 5.1287639337136135e-05, 'test_acc': 14.28, 'test_loss': 3.3309000898449225e-05, 'consensus_test': 23.04, 'iteration': 499}]), (3, [{'train_acc': 13.411341134113412, 'train_loss': 4.8316928699147325e-05, 'test_acc': 13.98, 'test_loss': 3.304229994231909e-05, 'consensus_test': 23.04, 'iteration': 499}]), (4, [{'train_acc': 20.312031203120313, 'train_loss': 5.2206743474387606e-05, 'test_acc': 19.92, 'test_loss': 3.193969205883586e-05, 'consensus_test': 23.04, 'iteration': 499}]), (5, [{'train_acc': 20.927092709270926, 'train_loss': 4.486035617484903e-05, 'test_acc': 20.91, 't

In [10]:
torch.where(a == b, torch.Tensor([5]), torch.Tensor([0]))

tensor([0., 5., 0.])